In [20]:
#Importing libraries

import json
import csv
import matplotlib
import numpy as np
from os import listdir
from sklearn import linear_model
import random
import copy

In [2]:
# The order of La liga at the end of the season
#The idea is to use this to predict the result of the matches, ie Barcelona won all of them and levante won none
order ={}
order[1] = 'barcelona'
order[2] = 'real madrid'
order[3] = 'atl madrid'
order[4] = 'villarreal'
order[5] = 'ath bilbao'
order[6] = 'celta vigo'
order[7] = 'sevilla'
order[8] = 'malaga-cf'
order[9] = 'real sociedad'
order[10] = 'real betis'
order[11] = 'las palmas'
order[12] = 'valencia'
order[13] = 'espanyol'
order[14] = 'sd-eibar'
order[15] = 'deportivo'
order[16] = 'granada'
order[17] = 'sporting gijon'
order[18] = 'rayo vallecano'
order[19] = 'getafe'
order[20] = 'levante'

#getting the position of the team
def get_position(team):
    min_lev = [30, 99] #position, distance lev
    for key,t in order.iteritems():
        dist = lev(team,t)
        if dist<= min_lev[1]:
            min_lev[0] = key
            min_lev[1] = dist
    return min_lev[0]
    

#get_position('espanol')


In [3]:
#Consider that maybe instead of computing the distance between strings would be better to match words?
#Or other kinds of similarity computation

#this is necessary because of the accents an other errors from scrapping around
def levenshtein(s1, s2):
    #https://en.wikipedia.org/wiki/Levenshtein_distance
    if len(s1) < len(s2):
        return lev(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j previous_row and current_row are 1 char longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[-1]


def lev(s1,s2):
    #convert to lower everything to avoid mistakes
    return levenshtein(s1.lower(),s2.lower())

In [4]:
#Reading the bets and other info
bets = np.genfromtxt('/home/roc/workspace/fifa-db-players/match-results/1516.csv', delimiter=',', skip_header=1
                                ,dtype="|S10")

#matches
matches = bets[:,2:4]
aux = bets[:,6]
results = []
for r in aux:
    if r == 'D':
        results.append(0)
    elif r == 'A':
        results.append(1)
    else: #H
        results.append(-1)


In [5]:
#Result match my encoding
#homies wins -1
#tie 0
#outer wins 1

my_results_basic = []
for match in matches:
    p1 = get_position(match[0])
    p2 = get_position(match[1])

    #with only win/lose it gets 1% improvement
    if p1+1 == p2 or p2+1==p1:
        my_results_basic.append(0)
    elif p1<p2:
        my_results_basic.append(-1)
    else:
        my_results_basic.append(1)


def get_accuracy(results, my_results_basic):
    count = 0.0
    for i in range(len(results)):
        if results[i] == my_results_basic[i]:
            count +=1
    return count/len(results)*100.0
print "Based on the final Liga order accuracy", get_accuracy(results, my_results_basic),'%'

Based on the final Liga order accuracy 48.4210526316 %


In [6]:
#Get the prediction from the different betting webs

#returns the list witht the bets results
def get_results_from_bets(bets):
    results = []
    for bet in bets:
        if(bet[0] < bet[1] and bet[0] < bet[2]) :
            results.append(-1)
        elif(bet[1] < bet[0] and bet[1] < bet[2]) :
            results.append(0)
        elif(bet[2] < bet[0] and bet[2] < bet[1]) :
            results.append(1)
        else:
            results.append(666666)
    return results

for i in range(13):
    print 'bet odds', get_accuracy(results, get_results_from_bets(bets[:,22+i*3:25+i*3])),'%'


bet odds 54.2105263158 %
bet odds 55.0 %
bet odds 52.8947368421 %
bet odds 54.2105263158 %
bet odds 54.4736842105 %
bet odds 53.6842105263 %
bet odds 54.4736842105 %
bet odds 25.5263157895 %
bet odds 30.2631578947 %
bet odds 21.8421052632 %
bet odds 40.2631578947 %
bet odds 25.2631578947 %
bet odds 38.6842105263 %


In [7]:
#Getting the club files
clubs = []
c_names = listdir('/home/roc/workspace/fifa-db-players/club-data')
for key, value in order.iteritems():
    min_lev = [30, 99] #position, distance lev
    for c in c_names:
        dist = lev(value,c[:-5])
        if dist<= min_lev[1]:
            min_lev[0] = c
            min_lev[1] = dist
    clubs.append(min_lev[0])

clubs_data = {}

#gathering the players info
for club in clubs:
    with open('/home/roc/workspace/fifa-db-players/club-data/'+club) as data_file:    
        clubs_data[club[:-5]] = json.load(data_file)


In [29]:
#Crete list with the training set with 80%
#Create list with the testing set with 20%
#No validation set
random.seed(5)

testing_matches = []
testing_results = []
training_matches = matches.tolist() #dont ask why this two are different
training_results = results[:]

for i in range(int(len(matches)*0.2)):
    t = random.randint(0,len(training_matches)-1)
    testing_matches.append(training_matches[t])
    testing_results.append(training_results[t])
    training_matches.pop(t)
    training_results.pop(t)

print len(training_matches), len(training_results), len(testing_matches), len(testing_results), len(matches)



304 304 76 76 380


In [17]:
#The function that executes the algorithm

def predict(func,playersA, playersB):
    return func(playersA, playersB)


In [18]:
def linear(playersA, playersB):
    #todo
    return result
# try for range -1 to 1
# try for the actual score

a


In [22]:
def decision_tree(playersA, playersB):
    #todo
    return result

In [ ]:
def random_forest(playersA, playersB):
    #todo
    return result

In [ ]:
def svn(playersA, playersB):
    #todo
    return result

In [23]:
def GP(playersA, playersB):
    #todo
    return result

In [ ]:
def neural_networks(playersA, playersB):
    #todo
    return result

In [ ]:
Dimensionality Reduction Algorithms
Gradient Boost & Adaboost

In [ ]:
#check what would happen if I betted 1€ for every winning team as the bet predicted